# Análisis Exploratorio de Datos - Encuesta CASEN

Este notebook tiene como objetivo explorar una base de datos CASEN en formato `.dta`, identificar variables relevantes para el estudio de **migración y salud**, y generar un subset para análisis posteriores.

## Importar librerias
Inicialmente importamos las librerias necesarias para el funcionamiento del notebook.

In [1]:
import pandas as pd

## Lectura archivo CASEN
Se debe el archivo correspondiente a los datos de la encuesta CASEN usando pandas.<br>
Este archivo se obtiene desde el siguiente [enlace](https://observatorio.ministeriodesarrollosocial.gob.cl/encuesta-casen-2022).<br>
El manual del funcionamiento de la encuesta CASEN y los codigos que los componen se encuentra en el siguiente [Enlace](https://observatorio.ministeriodesarrollosocial.gob.cl/storage/docs/casen/2017/Libro_de_Codigos_Casen_2017.pdf).

In [4]:
# Cargar archivo .dta (CASEN)
ruta = '../../data/raw_data/raw_casen.dta' 

# Leer el DTA
df = pd.read_stata(ruta, convert_categoricals=False)

# Mostrar las primeras filas
df.head()

C:\Users\Alan\AppData\Local\Temp\ipykernel_164912\2604134342.py:5: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  df = pd.read_stata(ruta, convert_categoricals=False)


,id_vivienda,folio,id_persona,region,area,cod_upm,nse,estrato,hogar,expr,...,men18c,may60c,tipohogar,tot_hog,ind_hacina,indsan,ten_viv,ten_viv_f,allega_ext,allega_int
0,1000901,100090101,1,16,2,10009,4,1630324,1,43,...,0,1,3,1,1,1,1,1,0,0
1,1000901,100090101,2,16,2,10009,4,1630324,1,43,...,0,1,3,1,1,1,1,1,0,0
2,1000901,100090101,3,16,2,10009,4,1630324,1,44,...,0,1,3,1,1,1,1,1,0,0
3,1000902,100090201,1,16,2,10009,4,1630324,1,51,...,1,1,5,1,1,2,1,1,0,1
4,1000902,100090201,2,16,2,10009,4,1630324,1,51,...,1,1,5,1,1,2,1,1,0,1


## Seleccion de columnas relevantes
Para nuestro estudio seleccionaremos las siguientes columnas:
* Estrato: Contiene informacion del codigo de comuna, area urbana o rural y nivel socioeconomico. Estrato viene de forma numerica comuna-area-nse(nivel socio economico) representadas por un numero los primeros valores indican el codigo comunal, el penultimo valor indica si es urbano o rural (1 o 2 -> urbano o rural) y el ultimo indica el nivel socio economico con valores de 1 a 7 indicando si es bajo a alto respectivamente.
* Region: indica la Region
* r1a: Indica si la persona es de nacionalidad chilena o extranjera
* r1a_esp: Especifica la nacionalidad si corresponde
* r1a_cod: Indica el codigo de la nacionalidad
* s13: estado de Salud de la persona al momento de la encuesta


In [5]:
# Selección manual de columnas clave
cols_interes = ['estrato', 'region', 'r1a', 'r1a_esp', 'r1a_esp_cod','s13']  
subset = df[cols_interes].copy()
subset.head()

,estrato,region,r1a,r1a_esp,r1a_esp_cod,s13
0,1630324,16,1,,NaN,1
1,1630324,16,1,,NaN,1
2,1630324,16,1,,NaN,1
3,1630324,16,1,,NaN,1
4,1630324,16,1,,NaN,1


## Exportacion a CSV encuesta CASEN
Finalmente exportamos a un csv la informacion obtenida de la encuesta CASEN

In [ ]:
# Guardar como CSV
subset.to_csv('../../data/procesed_data/casen_subset.csv', index=False)